In [1]:
import setGPU0
import os
from io_functions import *
from draw_functions import *
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
#from keras.utils.visualize_util import plot, model_to_dot
from IPython.display import SVG
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from niki_visual import *

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)
Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.
using gpu0


In [2]:
def build_model(num_branch=0, conv_per_branch=1, filters=3, num_dense=1,
                num_nodes=10, dropout=0.5, patience=5):
    # Make the list of models
    models = []
    
    # Initialize the models (model is the central, final, dense, merged one)
    inputs = Input(shape=(1, 20, 20, 25), name='Input Layer')
      
    # Add in the convolutional branches and layers (if there are any)
    if num_branch: # (check if there are any convolutional layers)
        for i in range(num_branch):
            for j in range(conv_per_branch):
                # Default 3,3,4; increase based on branch/depth (I think this is the best way to do this?)
                a, b, c = 3+i+j, 3+i+j, 4+i+j
                # Only specify shape if it's the first layer of the branch
                if not j:
                    x = Convolution3D(filters, a, b, c,
                                      input_shape = (1, 20, 20, 25), activation='relu',
                                      name=str((a,b,c))+' 3D Conv, B%s'%i+' L%s'%j)(inputs)
                else:
                    x = Convolution3D(filters, a, b, c,
                                      activation='relu', name=str((a,b,c))+' 3D Conv, B%s'%i+' L%s'%j)(x)
            x = MaxPooling3D(name='MaxPooling3D, B%s'%i)(x)
            models.append(Flatten(name='Flatten, B%s'%i)(x))
            
        # Combine the branches
        x = merge(models, mode='concat', name='Merge Layer')
                              
        # Add in the dense layers
        for i in range(num_dense):
            x = Dense(num_nodes, activation='relu', name='Dense, '+'N%s '%num_nodes+'L%s'%i)(x)
            x = Dropout(dropout, name='Dropout of %s'%dropout)(x)
    else: # (no convolutional layers)
        # Add in the dense layers
        for i in range(num_dense):
            x = Dense(num_nodes, activation='relu', input_dim=10000, name='Dense, '+'N%s '%num_nodes+'L%s'%i)(x)
            x = Dropout(dropout, name='Dropout of %s'%dropout)(x)
    
    # Add output layer, compile, and return it
    x = Dense(1, init='uniform', activation='sigmoid', name='Binary Output Layer')(x)
    model = Model(input=inputs, output=x)
    model.compile(loss='binary_crossentropy', optimizer='sgd')
    return model

In [3]:
# Returns the loss given certain definining parameters
def branin(num_branch, conv_per_branch, filters, num_dense, num_nodes, dropout, patience):
    # Get the data
    if conv_per_branch!=0:
        train_data, test_data, train_labels, test_labels = train_test(shape=(1, 20, 20, 25), split=0.33)
    else:
        train_data, test_data, train_labels, test_labels = train_test(shape=(10000,), split=0.33)
    
    # Build the model
    model = build_model(num_branch, conv_per_branch, filters, num_dense, num_nodes, dropout, patience)
    #check = ModelCheckpoint(filepath="./tmp/test%s.hdf5"%num_dense, verbose=1)
    early = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

    # Train the network
    my_fit = model.fit(train_data, train_labels, nb_epoch=10000, validation_split=0.2,
                   batch_size=1000, verbose=1, callbacks=[early])#[check, early])

    # Get the area under the curve for ROC curve on test set
    pred = predictions[label][0].reshape(predictions[label][0].shape[0])
    truth = predictions[label][1].astype(int)
    fpr, tpr, _ = roc_curve(truth, pred)
    roc_auc = auc(fpr, tpr)
    
    # Return the misclassification at 
    return float(roc_auc)

In [4]:
# Here is the main function
def main(job_id, params):
    print 'Anything printed here will end up in the output directory for job #%d' % job_id
    print params
    return branin(params['num_branch'], params['conv_per_branch'], params['filters'],
                  params['num_dense'], params['num_nodes'], params['dropout'], params['patience'])